In [ ]:
import librosa
import keras
from keras.models import Model
from keras.layers import Conv2D,MaxPool2D,BatchNormalization,Input,Activation,MaxPool1D,Conv1D
from keras.layers import CuDNNGRU as GRU
from keras.layers import CuDNNLSTM as LSTM
from keras.layers import Reshape, Bidirectional,Flatten,Dense,InputLayer,Permute,Multiply

In [1]:
import tensorflow as tf
import numpy as np
import tqdm
import glob
import os

In [2]:
data_root = "/home/philip/Files/Keyword_spot/"

In [ ]:
class_indices = {'bed': 0, 'bird': 1, 'cat': 2, 'dog': 3, 'down': 4, 'eight': 5, 'five': 6, 'four': 7, 'go': 8, 'happy': 9, 
                 'house': 10, 'left': 11, 'marvin': 12, 'nine': 13, 'no': 14, 'off': 15, 'on': 16, 'one': 17, 'right': 18,
                 'seven': 19, 'sheila': 20, 'silence': 21, 'six': 22, 'stop': 23, 'three': 24, 'tree': 25, 'two': 26,
                 'up': 27, 'wow': 28, 'yes': 29, 'zero': 30}

In [4]:
load_audio = True
def get_label_from_filename(full_filename):
    dirname = os.path.dirname(full_filename)
    label = dirname.split("/")[-1]
    if label == 'silence_bg':
        label ='silence'
    label_indice = class_indices[label]
    one_hot_encode = keras.utils.to_categorical(label_indice,len(class_indices))
    return one_hot_encode
def get_sequence_from_file(filename):
    y,sr = librosa.load(filename,sr=16000)
    if load_audio:
        return y
    else:
        mel = librosa.feature.melspectrogram(y,sr,hop_length=256)
        db = librosa.power_to_db(mel, ref=np.min)
        return db

In [7]:
all_files = glob.glob(data_root+"train/audio/***/*.wav") + glob.glob(data_root+"train/silence/*.wav") + glob.glob(data_root+"train/silence_bg/*.wav")

In [8]:
def data_gen():
    while True:
        for file in all_files:
            label = get_label_from_filename(file)
            seq = get_sequence_from_file(fuil)
            yield seq, label

In [ ]:
attention = False
def conv_1d_model(inputs):
    x = Conv1D(128,512,padding='same',activation='relu', dilation_rate=4)(inputs)
    x = BatchNormalization()(x)
    x = MaxPool1D(pool_size=(20),strides=10)(x)
    x = Conv1D(256,256,padding='same', activation='relu',dilation_rate=4)(x)
    x = BatchNormalization()(x)
    x = MaxPool1D(pool_size=(20),strides=10)(x)
    x = Bidirectional(LSTM(256,return_sequences=True),merge_mode='ave')(x)
    x = Bidirectional(LSTM(256,return_sequences=True),merge_mode='ave')(x)        
    if attention:
        a = Permute((2,1))(x)
        a = Dense(158, activation='softmax',input_dim=(158,))(a)
        a = Permute((2,1))(a)
        x = Multiply()([x,a])
    x = MaxPool1D(158)(x)
    o = Flatten()(x)
    o = Dense(31,activation='softmax')
    return o

In [9]:
input_layer = Input(16000,1)

KeyboardInterrupt: 